<center>
    <p style="text-align:center">
    <img alt="arize logo" src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="300"/>
        <br>
        <a href="https://docs.arize.com/arize/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/client_python">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-11t1vbu4x-xkBIHmOREQnYnYDH1GDfCg">Community</a>
    </p>
</center>

<center><h1>Using Arize with Langchain 🦜🔗 </h1></center>

LangChain is a framework that helps you prototype LLM applications quickly. Use Arize and LangChain together to trace, evaluate, and iterate on your LLM apps and agents.

### Running This Notebook
1. Step through each section below, pressing play on the code blocks to run the cells.
2. Log in your browser to the Arize App
3. Copy and paste your Arize Space ID and API key

# Step 0. Install Dependencies

Install Langchain and Arize packages.

⚠️ Use a GPU to save time generating embeddings. Click on 'Runtime', select 'Change Runtime Type' and
select 'GPU'.

In [1]:
!pip install -q langchain langchain_community langchain-openai

!pip install -q "arize[AutoEmbeddings]" "arize-otel>=0.7.0" "openinference-instrumentation-langchain>=0.1.4"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 12.5 MB/s eta 0:00:00
ERROR: pip's de

# Step 1. Setup Tracing

Copy the Arize `API_KEY` and `SPACE_ID` from your Space Settings page (shown below) to the variables in the cell below.

<center><img src="https://storage.googleapis.com/arize-assets/barcelos/Screenshot%202024-11-11%20at%209.28.27%E2%80%AFPM.png" width="700"></center>



In [2]:
# Import open-telemetry dependencies
from arize.otel import register
from getpass import getpass

# Setup OTEL via our convenience function
tracer_provider = register(
    space_id=getpass("Enter your Arize Space ID:"),
    api_key=getpass("Enter your Arize API Key:"),
    project_name="langchain-tracing",
)

# Import the automatic instrumentor from OpenInference
from openinference.instrumentation.langchain import LangChainInstrumentor

# Finish automatic instrumentation
LangChainInstrumentor().instrument(tracer_provider=tracer_provider)

Enter your Arize Space ID:··········
Enter your Arize API Key:··········
🔭 OpenTelemetry Tracing Details 🔭
|  Arize Project: langchain-tracing
|  Span Processor: BatchSpanProcessor
|  Collector Endpoint: otlp.arize.com
|  Transport: gRPC
|  Transport Headers: {'authorization': '****', 'api_key': '****', 'arize-space-id': '****', 'space_id': '****', 'arize-interface': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [3]:
import os

os.environ["OPENAI_API_KEY"] = getpass("Your OpenAI API Key: ")

Your OpenAI API Key: ··········


## Step 2: Define LLM

In [4]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0, verbose=True)

## Step 3: Test LLM Responses and Logging into Arize
Use some simple prompts to test if the LLM works properly and each prompt-response pair is logged into Arize with embeddings.

In [5]:
llm_result = llm.generate(
    [
        "Tell me an interesting fact about pandas.",
        "Explain the concept of overfitting in 2 sentences.",
    ]
)

## Step 4: Test LLM Chain and Agents with Arize Integration

In [6]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.
Title: {title}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.
Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(
    input_variables=["synopsis"], template=template
)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

overall_chain = SimpleSequentialChain(
    chains=[synopsis_chain, review_chain], verbose=True
)

test_prompts = [
    {
        "input": "documentary about pandas who are about be extinct because of global warming"
    },
    {"input": "once upon a time in hollywood"},
    {"input": "the best mo observability tooling"},
]
overall_chain.apply(test_prompts)

/tmp/ipython-input-1806936880.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)
/tmp/ipython-input-1806936880.py:30: LangChainDeprecationWarning: The method `Chain.apply` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~batch` instead.
  overall_chain.apply(test_prompts)




> Entering new SimpleSequentialChain chain...
 "Panda Planet: A Race Against Time"

In a world where global warming is rapidly changing the environment, the beloved pandas are facing a dire threat of extinction. As their natural habitats are destroyed and their food sources dwindle, a team of dedicated scientists and conservationists embark on a mission to save these gentle creatures from disappearing forever.

The play follows the journey of this team as they travel to remote areas of China to study and document the pandas' behavior and living conditions. Through their research, they uncover the devastating effects of global warming on the pandas and the urgent need for action.

As they race against time to find a solution, the team faces challenges and obstacles, both from nature and from those who do not believe in the urgency of the situation. But their determination and passion for these animals drive them forward, as they strive to raise awareness and make a difference before i

[{'input': 'documentary about pandas who are about be extinct because of global warming',
  'output': '\n\n"Panda Planet: A Race Against Time" is a timely and poignant play that tackles the urgent issue of global warming and its impact on our planet\'s wildlife. Through a captivating story and stunning visuals, the audience is taken on a journey to the remote areas of China where the beloved pandas are facing extinction.\n\nThe play\'s strong ensemble cast brings to life a team of dedicated scientists and conservationists who are determined to save these gentle creatures from disappearing forever. As they face challenges and obstacles, both from nature and from those who do not believe in the urgency of the situation, the audience is drawn into their emotional and thought-provoking journey.\n\nThe play\'s message is clear and powerful, highlighting the devastating effects of global warming on the pandas and the urgent need for action. Through their research and determination, the team 